In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"

In [5]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, transforms=[ResizeToDivisibleBy32()]
)
print(len(labeled_dataset))

data/INRIAAerialImageLabellingDataset\train
180


In [6]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH, split="test", transforms=[ResizeToDivisibleBy32()]
)
print(len(test_dataset))

data/INRIAAerialImageLabellingDataset\test
144


## Sanity check data

In [7]:
train_loader = DataLoader(labeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [9]:
to_pil_transform = transforms.ToPILImage()

In [10]:
img = to_pil_transform(images.squeeze())

In [11]:
# img.show()

In [12]:
msk = to_pil_transform(masks.squeeze())

In [13]:
# msk.show()

## Run example model on single image

In [14]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
).to(device)

In [15]:
baseline_model = BaselineModel()

In [16]:
torch.cuda.empty_cache()

In [17]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 5024, 5024])
torch.Size([1, 1, 5024, 5024])


In [18]:
print(output.shape)

torch.Size([1, 1, 5024, 5024])


In [19]:
print(output_baseline.shape)

torch.Size([1, 1, 5024, 5024])


In [20]:
outp = to_pil_transform(output.squeeze().cpu().numpy())

In [21]:
unique, counts = np.unique(outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 25240057, 255: 519}


In [22]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 25240057, 1.0: 519}


In [23]:
# outp.show()

In [24]:
rand_outp = to_pil_transform(output_baseline.squeeze().cpu().numpy())

In [25]:
unique, counts = np.unique(rand_outp, return_counts=True)
print(dict(zip(unique, counts)))

{0: 12621252, 255: 12619324}


In [26]:
# rand_outp.show()

In [27]:
evaluate_result(output, masks)

{'iou': tensor(2.9733e-05),
 'f1': tensor(5.9464e-05),
 'accuracy': tensor(0.9267),
 'recall': tensor(0.0329)}

In [28]:
evaluate_result(output_baseline, masks)

{'iou': tensor(0.0683),
 'f1': tensor(0.1278),
 'accuracy': tensor(0.5000),
 'recall': tensor(0.5166)}

In [29]:
evaluate_result(masks, masks)

{'iou': tensor(1.),
 'f1': tensor(1.),
 'accuracy': tensor(1.),
 'recall': tensor(1.)}

## Run example model on dataset

In [30]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [33]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = (output > 0.5).float()

        iter_metrics = evaluate_result(output, masks)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [ ]:
print(metrics_dict)

{'iou': [tensor(7.7134e-07)], 'f1': [tensor(1.5427e-06)], 'accuracy': [tensor(0.8459)], 'recall': [tensor(1.0581e-06)]}
